In [1]:
%run HelperFuncs.ipynb

In [2]:
# IR CLASSES


class Channel:
    def __init__ (self, src_act, dst_act,  src, dst, init_token = 0):
        self.name = src+"_to_"+dst
        self.src_port = src
        self.src_act = src_act
        self.dst_port = dst
        self.dst_act = dst_act
        self.init_token = init_token
        self.size = 1
        self.datatype = False
    def get_size(self):
        if self.datatype:
            return self.datatype.size()
        return self.size
    def __str__(self):
        return "<channel name='"+self.name+"' srcActor='"+self.src_act+"' srcPort='"+self.src_port+"' dstActor='"+self.dst_act+"' dstPort='"+self.dst_port+"' size='"+str(self.get_size())+ "' dtype='"+str(self.datatype)+ "' initialTokens='"+str(self.init_token)+"'/>"
    def add_dt(self, dt):
        self.datatype = dt
    def get_dt(self):
        return self.datatype

class Port:
    def __init__ (self, direction, name, rate):
        self.name = name
        self.direction = direction
        self.rate = rate
    def __str__(self):
        return "<port type='"+self.direction+"' name='"+self.name+"' rate='"+','.join(map(str,self.rate))+"'/>"

def get_channel(src, dst):
    return Channel(src.name.split("_")[0], dst.name.split("_")[0], src.name, dst.name)
# INPUT: list of dicts of ports - [{'dst' : Port, 'src' : Port}]
# OUTPUT: list of channels - [Channel]
def get_channels(ls):
    return [get_channel(x["src"], x["dst"]) for x in ls]

class CSDFNode:
    def __init__():
        pass
    def add_dt(self, dt):
        self.datatype = dt
    def get_dt(self):
        return self.datatype
    def has_dt(self):
        try:
            if self.datatype == True:
                return True
            else:
                return True
        except:
            return False
        
class Param(CSDFNode):
    def __init__(self, name, label):
        self.name = name
        self.label = label
        self.input = Port('in', name + '_in', [1])
        self.out_ctr = 0
        self.output = []
    def new_outport(self, r = [1]):
        np = Port('out', self.name + "_x"+str(self.out_ctr)+ '_out', r)
        self.output.append(np)
        self.out_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Param' >\n" + '\n'.join(map(str, [self.input]+self.output)) + "\n</actor>"

class Get(CSDFNode):
    def __init__(self, name, idx):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
        self.idx = idx
    def __str__(self):
        return "<actor name='"+self.name+"' type='Get' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"

class Transpose(CSDFNode):
    def __init__(self, name):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Transpose' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"

    
class Dearray(CSDFNode):
    def __init__(self, name, portin, portout):
        self.name = name
        self.input = portin
        self.output = portout
    def __str__(self):
        return "<actor name='"+self.name+"' type='Dearray' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    

        
class Rearray(CSDFNode):
    def __init__(self, name, portin, portout, mstr):
        self.name = name
        self.input = portin
        self.output = portout
        self.masternode = mstr
    def __str__(self):
        return "<actor name='"+self.name+"' type='Rearray' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    

    
class Join(CSDFNode):
    def __init__(self, name):
        self.name = name
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Join' >\n" + '\n'.join(map(str, [self.input, self.output])) + "\n</actor>"
    
# Must have new value for every attachment
class Value(CSDFNode):
    def __init__(self, name, value, kind):
        self.name = name
        self.value = value
        self.kind = kind
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Value' value='"+self.value+"' kind='"+self.kind+"'>\n" + '\n'.join(map(str, self.output)) + "\n</actor>"

class Reduce(CSDFNode):
    def __init__(self, name, subfunc, initval, seq = False):
        self.name = name
        self.type =  "ReduceSeq" if seq else "Reduce"
        self.subfunc = subfunc
        self.initval = initval
        self.seq = seq
        self.rep = 1
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='"+self.type+"'>\n" + '\n'.join(map(str, [self.output, self.input])) + "\n</actor>"


class Map(CSDFNode):
    def __init__(self, name, subfunc):
        self.name = name
        self.subfunc = subfunc
        self.rep = 1
        self.input = Port('in', name + '_in', [1])
        self.output = Port('out', name + '_out', [1])
    def __str__(self):
        return "<actor name='"+self.name+"' type='Map'>\n" + '\n'.join(map(str, [self.output, self.input])) + "\n</actor>"
    
class Zip(CSDFNode):
    def __init__(self, name : str):
        self.name = name
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "_x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='Zip'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

class Mather(CSDFNode):
    def __init__(self, name, kind):
        self.name = name
        self.kind = kind
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "_x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='"+self.kind+"'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

class UserFun(CSDFNode):
    def __init__(self, name : str, label):
        self.name = name
        self.label = label
        self.input = []
        self.in_ctr = 0
        self.output = Port('out', name + '_out', [1])
    def new_inport(self):
        np = Port('in', self.name + "_x"+str(self.in_ctr)+ '_in', [1])
        self.input.append(np)
        self.in_ctr += 1
        return np
    def __str__(self):
        return "<actor name='"+self.name+"' type='UserFun'  label='"+self.label+"'>\n" + '\n'.join(map(str, [self.output] + self.input)) + "\n</actor>"

def getports(node):
    return (node.input if type(node.input) == list else [node.input]) + (node.output if type(node.output) == list else [node.output])
    

In [17]:
# TYPE CLASSES
class TYPECLASS:
    pass
class SV(TYPECLASS):
    def __init__(self, name,  val):
        self.label = name
        self.val = val
    def __str__(self):
        return self.label + ": " + str(val)
class Float(TYPECLASS):
    def __init__(self):
        pass
    def size(self):
        return 4
    def __str__(self):
        return "Float"
class Int(TYPECLASS):
    def __init__(self):
        pass
    def size(self):
        return 1
    def __str__(self):
        return "Int"
class Array(TYPECLASS):
    def __init__(self, length, subdata):
        self.length = length
        self.subdata = subdata
    def size(self):
        return self.length * self.subdata.size()
    def __str__(self):
        return "Array (" + str(self.length) + ", " + str(self.subdata) + ")"
class Tuple(TYPECLASS):
    def __init__(self, subdatals = []):
        self.subdatals = subdatals
    def length():
        return len(self.subdatals)
    def add_subdata(self, subdata):
        self.subdatals.append( subdata)
    def size(self):
        return sum([sd.size() for sd in self.subdatals])
    def __str__(self):
        return "Tuple (" + ", ".join([str(x) for x in self.subdatals] )+ ")"
class Unknown(TYPECLASS):
    def __init__(self):
        pass
    def size(self):
        return 40
    def __str__(self):
        return "Unknown"

In [30]:
x = Int()
x

In [34]:
isinstance(x, TYPECLASS)

True

In [24]:
x.__class__

__main__.Int

In [4]:
# Parser


# Input: Program code as string
def unpack_program(program):
    node_ctr = 0
    def getname():
        nonlocal node_ctr
        node_ctr += 1
        return "n" + str(node_ctr)
    
    named_nodes = {}
    
    starting_inputs = getmaininputs(program)
    sizevars = getsvs(program)
    main_func = commasplit(getfunc(program))[-1]
    
    def mk_bby_func(bby, n_inputs):
        assert(len(set(bby.strip()).intersection(set("{}()[],. "))) == 0)
        if bby in ["mult", "add"]:
            new_node = Mather(getname(), bby)
            i1 =  Param(getname(), "fst")
            i2 =  Param(getname(), "snd")
            return {
                "nodes" : [new_node, i1, i2],
                "channels" : [get_channel(i1.new_outport(), new_node.new_inport()),
                             get_channel(i2.new_outport(), new_node.new_inport())],
                "inputs" : [i1.input, i2.input],
                "output" : new_node.output
            }
            
        new_node = UserFun(getname(), bby)
        for i in range(n_inputs):
            new_node.new_inport()
        return {
            "nodes" : [new_node],
            "channels" : [],
            "inputs" : new_node.input,
            "output" : new_node.output
        }
        
    
    # Output: {nodes : [Node], channels : [{Port, Port}], inputs : [Port], output : Port}
    def unpack_func(func):
        nodes = []
        channels = []
        myinputs = []
        local_names = getnames(func) if "=>" in func else []

        for name in local_names:
            new_node = Param(getname(), name)
            nodes.append(new_node)
            myinputs.append(new_node.input)
            assert(name not in named_nodes)
            named_nodes[name] = new_node  
        
        # Input: body
        # does: add nodes + channels to func list
        # Output: {in : Port, out :Port}
        def unpack_body(body):
            comps = componentsplit(body)
            assert(len(comps) > 0)
            assert(len(comps) % 2 == 1)
            
            curr = unpack_comp(comps[0])
            for i in range(1, len(comps), 2):
                assert(comps[i] in ["o", "$", ":>>"])
                new = unpack_comp(comps[i+1])
                if comps[i] == ":>>":
                    assert(new["in"] and curr["out"])
                    channels.append({"dst":new["in"], "src" : curr["out"]})
                    curr["out"] = new.get("out")
                else:
                    assert(curr["in"] and new["out"])
                    channels.append({"dst" : curr["in"], "src" : new["out"]})
                    curr["in"] = new.get("in")
            return curr
            
        
        # Input: single component
        # does: add nodes + channels to func list
        # Output: {in : Port, out :Port}
        def unpack_comp(comp):
            assert(len(componentsplit(comp)) == 1)
            op = getNextOp(comp)
            if op in named_nodes.keys():
                assert(op == comp.strip())
                src = named_nodes[op]
                src_port = src.new_outport()
                return({"out" : src_port})
            if op == "Value":
                args = getNextParams(comp)
                # It's the final node
                assert(len(args) == 2)
                assert(all([len(componentsplit(arg)) == 1 for arg in args]))
                new_node = Value(getname(), args[0], args[1])
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Get":
                args = getNextParams(comp)
                assert(len(args) == 2)
                assert(all([len(componentsplit(arg)) == 1 for arg in args]))
                new_node = Get(getname(), args[1])
                incoming = unpack_body(args[0])["out"]
                channels.append({"src" : incoming, "dst" : new_node.input})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Transpose":
                assert(len(getNextParams(comp)) == 0)
                new_node = Transpose(getname())
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op == "Join":
                assert(len(getNextParams(comp)) == 0)
                new_node = Join(getname())
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op == "Zip":
                args = getNextParams(comp)
                new_node = Zip(getname())
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op in ["mult", "add"]:
                args = getNextParams(comp)
                new_node = Mather(getname(), op)
                for arg in args:
                    new_in = new_node.new_inport()
                    new_out = unpack_body(arg)["out"]
                    channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
            if op == "Map":
                args = getNextParams(comp)
                assert(len(args) == 1)
                f = args[0].strip()
                if f.startswith("VectorizeUserFun"):
                    f = getNextParams(f)[1]
                if isfunction(f):
                    child_func = unpack_func(getbc(f))
                elif '(' in f:
                    child_func = unpack_func(f)
                else:
                    child_func = mk_bby_func(f, 1)
                new_node = Map(getname(), child_func)
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            if op in ["Reduce", "ReduceSeq"]:
                args = getNextParams(comp)
                assert(len(args) == 2)
                f = args[0].strip()
                if f.startswith("VectorizeUserFun"):
                    f = getNextParams(f)[1]
                if isfunction(f):
                    child_func = unpack_func(getbc(f))
                elif '(' in f:
                    child_func = unpack_func(f)
                else:
                    child_func = mk_bby_func(f, 2)
                init_val = args[1]
                new_node = Reduce(getname(), child_func, init_val, op == "ReduceSeq")
                nodes.append(new_node)
                return({"in" : new_node.input, "out" : new_node.output})
            else:
                new_node = UserFun(getname(), op)
                if hasparams(comp):
                    for arg in getNextParams(comp):
                        new_in = new_node.new_inport()
                        new_out = unpack_body(arg)["out"]
                        channels.append({"src" : new_out, "dst" : new_in})
                nodes.append(new_node)
                return({"out" : new_node.output})
        
        
        mybod = getbody(func) if "=>" in func else func
        res = unpack_body(mybod)
        if "=>" in func:
            assert(not res.get("in"))
        else:
            myinputs = [res["in"]]
        myoutput = res["out"]
        
        for name in local_names:
            named_nodes.pop(name)
        
        return {
            "nodes" : nodes,
            "channels" : get_channels(channels),
            "inputs" : myinputs,
            "output" : myoutput
        }
            
    parsed = unpack_func(main_func)
    return {
        "sizevars" : sizevars,
        "inputs" : starting_inputs,
        "code" : getfunc(program),
        "graph" : parsed
    }       

In [5]:
def parse_file(fn):
    program = read_program(fn)
    p = unpack_program(program)
    return p

In [6]:
def test():
    return parse_file(add_cwd("highLevel/mmNN"))

In [8]:
p = test()

In [10]:
g = p['graph']
# printgraph_simple(g)

n1: Param
n2: Param
n15: Map
[('n1', 'n15')]

       Subgraph of Map node  n15
n3: Param
n13: Map
n14: Transpose
[('n14', 'n13')]
[('n2', 'n14')]

       Subgraph of Map node  n13
n4: Param
n6: Reduce
n11: Map
n12: Zip
[('n11', 'n6')]
[('n3', 'n12')]
[('n4', 'n12')]
[('n12', 'n11')]

       Subgraph of Reduce node  n6
n5: Mather

       Subgraph of Map node  n11
n7: Param
n9: Get
n10: Get
n8: Mather
[('n7', 'n9')]
[('n9', 'n8')]
[('n7', 'n10')]
[('n10', 'n8')]
